# Task 2 RNN

References:
- [ECG Heartbeat Classification: A Deep Transferable Representation](https://arxiv.org/pdf/1805.00794.pdf)

## Load dependencies

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical

## MIT-BIH  Arrhythmia Dataset

- Number of Samples: 109446
- Number of Categories: 5
- Sampling Frequency: 125Hz
- Data Source: Physionet's MIT-BIH Arrhythmia Dataset
- Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]
- Remark: All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 188.
- The final element of each row denotes the class to which that example belongs.


In [2]:
df_mitbih_train = pd.read_csv("../ecg_dataset/mitbih_train.csv", header = None)
df_mitbih_test = pd.read_csv("../ecg_dataset/mitbih_test.csv", header = None)

# print shapes of the dataframes
print("The shape of the mitbih_train is : ", df_mitbih_train.shape)
print("The shape of the mitbih_test is : ", df_mitbih_test.shape)

The shape of the mitbih_train is :  (87554, 188)
The shape of the mitbih_test is :  (21892, 188)


# Build model for classification

Configure training and testing sets. 

In [23]:
X_train, y_train = df_mitbih_train.iloc[: , :-1], df_mitbih_train.iloc[: , -1]

X_test, y_test = df_mitbih_test.iloc[:,:-1], df_mitbih_test.iloc[: , -1]

In [24]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [25]:
print(y_train.shape)
print(y_test.shape)

(87554, 5)
(21892, 5)


In [27]:
# Define the number of classes
num_classes = 5

# Create a 1D CNN model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
2737/2737 [==============================] - 30s 11ms/step - loss: 0.1933 - accuracy: 0.9461 - precision_8: 0.9509 - recall_8: 0.9406
Epoch 2/10
2737/2737 [==============================] - 30s 11ms/step - loss: 0.1022 - accuracy: 0.9709 - precision_8: 0.9729 - recall_8: 0.9693
Epoch 3/10
2737/2737 [==============================] - 31s 11ms/step - loss: 0.0788 - accuracy: 0.9772 - precision_8: 0.9785 - recall_8: 0.9761
Epoch 4/10
2737/2737 [==============================] - 30s 11ms/step - loss: 0.0648 - accuracy: 0.9804 - precision_8: 0.9816 - recall_8: 0.9794
Epoch 5/10
2737/2737 [==============================] - 30s 11ms/step - loss: 0.0542 - accuracy: 0.9837 - precision_8: 0.9845 - recall_8: 0.9828
Epoch 6/10
2737/2737 [==============================] - 30s 11ms/step - loss: 0.0469 - accuracy: 0.9855 - precision_8: 0.9862 - recall_8: 0.9848
Epoch 7/10
2737/2737 [==============================] - 31s 11ms/step - loss: 0.0396 - accuracy: 0.9877 - precision_8: 0.9883 - re

In [28]:
# Evaluate the model on the test set
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

685/685 [==============================] - 2s 3ms/step - loss: 0.0869 - accuracy: 0.9819 - precision_8: 0.9826 - recall_8: 0.9815
Loss: 0.08693964779376984
Accuracy: 0.9818655252456665
Precision: 0.9825772643089294
Recall: 0.9815000891685486
685/685 [==============================] - 2s 2ms/step


In [29]:
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Print classification report
report = classification_report(np.argmax(y_test, axis=1), y_pred)
print(report)

685/685 [==============================] - 2s 3ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18118
           1       0.85      0.78      0.81       556
           2       0.98      0.92      0.95      1448
           3       0.77      0.76      0.77       162
           4       0.99      0.99      0.99      1608

    accuracy                           0.98     21892
   macro avg       0.92      0.89      0.90     21892
weighted avg       0.98      0.98      0.98     21892

